## Data load

In [16]:
import os
import random

# root = '/usr/local/MATLAB/R2024b/toolbox/lidar/lidardata/lcc/vlp16'
# root = '/data/tcar_no_intensity'
root = '/data/velodyne'
# root = '/data/Tcar_hs'

img_files = sorted(os.listdir(os.path.join(root, 'Image')))
pcd_files = sorted(os.listdir(os.path.join(root, 'PCD')))

idx = random.randint(0, len(img_files)-1)

## RoI and plane PCD Visualization

In [5]:
import open3d as o3d
import numpy as np
from utils.util import show_point_cloud
import random

idx = random.randint(0, len(img_files)-1)

pcd = o3d.io.read_point_cloud(os.path.join(root, 'PCD', pcd_files[idx]))
show_point_cloud(pcd, scailing=False, intensity=False)

points = np.asarray(pcd.points)
valid_mask = np.isfinite(points).all(axis=1)
points = points[valid_mask]

valid_mask = np.isfinite(points).all(axis=1)
points = points[valid_mask]

# Set region of interest (ROI)
mask = np.ones(points.shape[0])
# velodyne
mask = np.logical_and(mask, points[:, 0] < 6)
mask = np.logical_and(mask, points[:, 0] > 1)
mask = np.logical_and(mask, points[:, 1] < 3)
mask = np.logical_and(mask, points[:, 1] > -3)        
mask = np.logical_and(mask, points[:, 2] > -0.5)

roi_points = points[mask]
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(roi_points)

# Segment the plane using RANSAC
plane_model, inliers = pcd.segment_plane(distance_threshold=0.03, ransac_n=3, num_iterations=10000)
[a, b, c, d] = plane_model
plane_points = np.asarray(pcd.points)[inliers]

pcd.points = o3d.utility.Vector3dVector(plane_points)
plane_normal = np.array([a, b, c]) / np.linalg.norm([a, b, c])
plane_point = np.mean(plane_points, axis=0)

plane_pcd = o3d.geometry.PointCloud()
plane_pcd.points = o3d.utility.Vector3dVector(plane_points)

show_point_cloud(plane_pcd, scailing=False, intensity=False)

## Save the data

In [15]:
import open3d as o3d
import numpy as np
from utils.util import show_point_cloud
from tqdm import tqdm
from PIL import Image

# ---------- MAIN PIPELINE ----------
img_path = '../data/Image'
pcd_path = '../data/PCD'
whole_pcd_path = '../data/WholePCD'

# Read the point cloud file
os.makedirs(os.path.join(img_path), exist_ok=True)
os.makedirs(os.path.join(pcd_path), exist_ok=True)
os.makedirs(os.path.join(whole_pcd_path), exist_ok=True)

for i in tqdm(range(1000)):
    idx = random.randint(0, len(img_files)-1)

    pcd = o3d.io.read_point_cloud(os.path.join(root, 'PCD', pcd_files[idx]))
    # show_point_cloud(pcd, scailing=True, intensity=False)

    points = np.asarray(pcd.points)
    valid_mask = np.isfinite(points).all(axis=1)
    points = points[valid_mask]
    np.save(f"{whole_pcd_path}/{idx:04d}.npy", points)

    valid_mask = np.isfinite(points).all(axis=1)
    points = points[valid_mask]

    # Set region of interest (ROI)
    mask = np.ones(points.shape[0])
    # velodyne
    mask = np.logical_and(mask, points[:, 0] < 6)
    mask = np.logical_and(mask, points[:, 0] > 1)
    mask = np.logical_and(mask, points[:, 1] < 3)
    mask = np.logical_and(mask, points[:, 1] > -3)        
    mask = np.logical_and(mask, points[:, 2] > -0.5)
    
    roi_points = points[mask]
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(roi_points)

    # Segment the plane using RANSAC
    plane_model, inliers = pcd.segment_plane(distance_threshold=0.03, ransac_n=3, num_iterations=10000)
    [a, b, c, d] = plane_model
    plane_points = np.asarray(pcd.points)[inliers]

    pcd.points = o3d.utility.Vector3dVector(plane_points)
    plane_normal = np.array([a, b, c]) / np.linalg.norm([a, b, c])
    plane_point = np.mean(plane_points, axis=0)

    plane_pcd = o3d.geometry.PointCloud()
    plane_pcd.points = o3d.utility.Vector3dVector(plane_points)
    # show_point_cloud(plane_pcd, scailing=False, intensity=False)

    np.save(f"{pcd_path}/{idx:04d}.npy", np.asarray(plane_pcd.points))
    img = Image.open(os.path.join(root, 'Image', img_files[idx])) 
    img.save(f'{img_path}/{idx:04d}.png')

100%|██████████| 1000/1000 [01:24<00:00, 11.87it/s]
